In [1]:
from scripts.pdf_to_text_reader_script import convert_ocr_pdf_to_text
from transformers import pipeline
import pandas as pd
import json


/home/shiro/miniconda3/envs/3101_proj/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Creation of JSONS

In [2]:
pdf_file = '/home/shiro/dsa3101/Datasets/ocr_esg/Health/NA/Pfizer_2022_ocr.pdf'
doc_json = convert_ocr_pdf_to_text(pdf_file)
doc_json

['N N (en) N = (ab) iS Ee oO SOcie J Environmental & Governance Re In This Report = ON oo ff W AG 82 Introduction A Letter from our Chairman & CEO A Letter from Our Lead Independent Director 2022 Progress and Highlights Our Approach to ESG Priority ESG Issues Impact Stories Tackling the Health Equity Gap Together Keeping Patients at the Center of Everything We Do Environment Climate Change Sustainable Medicines Social Innovation and Global Health Human Capital Governance Ethics, Transparency, and Quality Accountability Performance Environment Social Governance Appendix GRI Index SASB Index TCFD About This Report Independent Accountants’ Report What this report covers This report shares an overview of our Environmental, Social, and Governance (ESG) vision, achievements, and progress in the year from January 1, 2022 to December 31, 2022, including data on 32 key performance indicators (KPIs), as well as our path toward a sustainable life sciences future.',
 'For more information, please 

# Bert

In [3]:
pipe = pipeline("text-classification", model="nbroad/ESG-BERT")
res = pipe(doc_json)
res

Device set to use cuda:0


[{'label': 'Business_Model_Resilience', 'score': 0.2287348508834839},
 {'label': 'Management_Of_Legal_And_Regulatory_Framework',
  'score': 0.399785578250885},
 {'label': 'Business_Model_Resilience', 'score': 0.5959138870239258},
 {'label': 'Human_Rights_And_Community_Relations', 'score': 0.300252228975296},
 {'label': 'Business_Model_Resilience', 'score': 0.4655517637729645},
 {'label': 'Access_And_Affordability', 'score': 0.3112299144268036},
 {'label': 'Employee_Health_And_Safety', 'score': 0.7517262101173401},
 {'label': 'Human_Rights_And_Community_Relations',
  'score': 0.23952540755271912},
 {'label': 'Access_And_Affordability', 'score': 0.6096283793449402},
 {'label': 'Human_Rights_And_Community_Relations',
  'score': 0.8588701486587524},
 {'label': 'Physical_Impacts_Of_Climate_Change', 'score': 0.9421326518058777},
 {'label': 'Human_Rights_And_Community_Relations',
  'score': 0.20805077254772186},
 {'label': 'Employee_Engagement_Inclusion_And_Diversity',
  'score': 0.8861854076

### Dummy Data Creation to create labels that are esg_cat - esg_subcat => Mimick our real labels

In [4]:
esg_categories = {
    "Business_Ethics": "Governance",
    "Management_Of_Legal_And_Regulatory_Framework": "Governance",
    "Business_Model_Resilience": "Governance",
    "Human_Rights_And_Community_Relations": "Social",
    "Access_And_Affordability": "Social",
    "Employee_Health_And_Safety": "Social",
    "Employee_Engagement_Inclusion_And_Diversity": "Social",
    "Critical_Incident_Risk_Management": "Social",
    "Product_Quality_And_Safety": "Social",
    "Supply_Chain_Management": "Social",
    "Labor_Practices": "Social",
    "Customer_Privacy": "Social",
    "Selling_Practices_And_Product_Labeling": "Social",
    "GHG_Emissions": "Environmental",
    "Physical_Impacts_Of_Climate_Change": "Environmental",
    "Energy_Management": "Environmental",
    "Waste_And_Hazardous_Materials_Management": "Environmental",
    "Water_And_Wastewater_Management": "Environmental",
    "Ecological_Impacts": "Environmental",
    "Air_Quality": "Environmental",
    "Director_Removal": "Governance",
    "Systemic_Risk_Management": "Governance",
    "Product_Design_And_Lifecycle_Management": "Governance",
    "Competitive_Behavior": "Governance"
}
df = pd.read_csv('/home/shiro/dsa3101/example_db_data/example_db_data pfizer_2024.csv') #data was generated using esg_bert on the pfizer => esg-bert labels not our actual model
df3 = df.copy()
df3['cat'] = df3['label'].map(esg_categories)
df3

df3['label'] = df3['cat'] + ' - ' + df3['label']
df3.rename(columns={'0': 'sentence'}, inplace=True)
df3[['sentence','label','score']].to_csv('eg2.csv')

In [5]:
df3

,Unnamed: 0,sentence,label,score,cat
0,0,(ab) iS Ee oO SOcie J Environmental & Gove...,Governance - Business_Ethics,0.256791,Governance
1,1,"For more information, please see About this Re...",Governance - Management_Of_Legal_And_Regulator...,0.399786,Governance
2,2,Where to find more information 14} 2022 Annua...,Governance - Business_Model_Resilience,0.760010,Governance
3,3,Our focus is rooted in our purpose to deliver...,Social - Human_Rights_And_Community_Relations,0.300252,Social
4,4,Stepping Up to Meet the Demands of a Changing...,Social - Access_And_Affordability,0.196161,Social
...,...,...,...,...,...
1328,1328,"In our opinion, management's assertion that th...",Governance - Management_Of_Legal_And_Regulator...,0.931985,Governance
1329,1329,"KPWGs LLP New York, New York March 16, 2023 ...",Social - Selling_Practices_And_Product_Labeling,0.642459,Social
1330,1330,The products discussed herein may have differ...,Social - Selling_Practices_And_Product_Labeling,0.820957,Social
1331,1331,Copyright © 2022-2023 Pfizer Inc.,Governance - Management_Of_Legal_And_Regulator...,0.323478,Governance


In [6]:
import json
res_dummy =df3[['label','score']].to_json(orient='records')
res_dummy = json.loads(res_dummy)

## Pretend our res pipe output is the data i imported converted to pd

In [7]:
pd.DataFrame(res_dummy)

,label,score
0,Governance - Business_Ethics,0.256791
1,Governance - Management_Of_Legal_And_Regulator...,0.399786
2,Governance - Business_Model_Resilience,0.760010
3,Social - Human_Rights_And_Community_Relations,0.300252
4,Social - Access_And_Affordability,0.196161
...,...,...
1328,Governance - Management_Of_Legal_And_Regulator...,0.931985
1329,Social - Selling_Practices_And_Product_Labeling,0.642459
1330,Social - Selling_Practices_And_Product_Labeling,0.820957
1331,Governance - Management_Of_Legal_And_Regulator...,0.323478


In [8]:
doc_json #list of sentences
df = pd.DataFrame(doc_json, columns=['sentence'])
df[['label','score']] = pd.DataFrame(res_dummy)
df #this is our example output from bert and to join it as a df

,sentence,label,score
0,N N (en) N = (ab) iS Ee oO SOcie J Environment...,Governance - Business_Ethics,0.256791
1,"For more information, please see About this Re...",Governance - Management_Of_Legal_And_Regulator...,0.399786
2,Where to find more information 14} 2022 Annual...,Governance - Business_Model_Resilience,0.760010
3,Our focus is rooted in our purpose to deliver ...,Social - Human_Rights_And_Community_Relations,0.300252
4,Introduction > Impact Stories > Environment > ...,Social - Access_And_Affordability,0.196161
...,...,...,...
1331,"In our opinion, management's assertion that th...",Governance - Management_Of_Legal_And_Regulator...,0.323478
1332,"KPWGs LLP New York, New York March 16, 2023 Ap...",Social - Customer_Privacy,0.120179
1333,The products discussed herein may have differe...,NaN,NaN
1334,Copyright © 2022-2023 Pfizer Inc.,NaN,NaN


# Data Confidence Scoring Filter

assuming Governance - Business_Ethic is output labels -> We first split it into esg_cat and sub_cat

In [9]:
df['label'][0].split(' - ')
df[['esg_cat','esg_subcat']] = df['label'].str.split(' - ',expand=True)
df = df [['sentence','esg_cat','esg_subcat','score']]

In [10]:
df

,sentence,esg_cat,esg_subcat,score
0,N N (en) N = (ab) iS Ee oO SOcie J Environment...,Governance,Business_Ethics,0.256791
1,"For more information, please see About this Re...",Governance,Management_Of_Legal_And_Regulatory_Framework,0.399786
2,Where to find more information 14} 2022 Annual...,Governance,Business_Model_Resilience,0.760010
3,Our focus is rooted in our purpose to deliver ...,Social,Human_Rights_And_Community_Relations,0.300252
4,Introduction > Impact Stories > Environment > ...,Social,Access_And_Affordability,0.196161
...,...,...,...,...
1331,"In our opinion, management's assertion that th...",Governance,Management_Of_Legal_And_Regulatory_Framework,0.323478
1332,"KPWGs LLP New York, New York March 16, 2023 Ap...",Social,Customer_Privacy,0.120179
1333,The products discussed herein may have differe...,NaN,NaN,NaN
1334,Copyright © 2022-2023 Pfizer Inc.,NaN,NaN,NaN


### Note = We need to keep this data in a dataframe first, we only insert into ESG_BERT after esg_llm insertion as we still dont have company name

In [11]:
df2 = df[df['score'] > 0.8]
df2

,sentence,esg_cat,esg_subcat,score
9,2022—like the two years preceding it—was punct...,Social,Human_Rights_And_Community_Relations,0.858870
10,"At the same time, continued climate change has...",Environmental,Physical_Impacts_Of_Climate_Change,0.942133
12,"At Pfizer, our commitment is anchored to our s...",Social,Employee_Engagement_Inclusion_And_Diversity,0.886185
16,4 Pfizer 2022 ESG Report Introduction to refin...,Governance,Management_Of_Legal_And_Regulatory_Framework,0.952679
17,Acting Ethically Our approach to helping addre...,Social,Human_Rights_And_Community_Relations,0.913696
...,...,...,...,...
1309,Should known or unknown risks or uncertainties...,Governance,Management_Of_Legal_And_Regulatory_Framework,0.903547
1322,Our examination was conducted in accordance wi...,Governance,Business_Ethics,0.974460
1324,An examination involves performing procedures ...,Governance,Business_Ethics,0.922007
1328,82 Pfizer 2022 ESG Report Our examination was ...,Governance,Management_Of_Legal_And_Regulatory_Framework,0.931985


In [12]:
from scripts.pdf_table_extract import extract_tables

tabular = extract_tables('/home/shiro/dsa3101/Datasets/ocr_esg/Health/NA/Pfizer_2022_ocr.pdf')

In [13]:
from scripts.prompts import prompts
from scripts.json_clean import json_clean
#from scripts.ask_llm import ask_llm

In [14]:
data = df2[['sentence','esg_cat','esg_subcat']]

In [15]:
data

,sentence,esg_cat,esg_subcat
9,2022—like the two years preceding it—was punct...,Social,Human_Rights_And_Community_Relations
10,"At the same time, continued climate change has...",Environmental,Physical_Impacts_Of_Climate_Change
12,"At Pfizer, our commitment is anchored to our s...",Social,Employee_Engagement_Inclusion_And_Diversity
16,4 Pfizer 2022 ESG Report Introduction to refin...,Governance,Management_Of_Legal_And_Regulatory_Framework
17,Acting Ethically Our approach to helping addre...,Social,Human_Rights_And_Community_Relations
...,...,...,...
1309,Should known or unknown risks or uncertainties...,Governance,Management_Of_Legal_And_Regulatory_Framework
1322,Our examination was conducted in accordance wi...,Governance,Business_Ethics
1324,An examination involves performing procedures ...,Governance,Business_Ethics
1328,82 Pfizer 2022 ESG Report Our examination was ...,Governance,Management_Of_Legal_And_Regulatory_Framework


In [16]:
tabular

['{"Unnamed: 0":{"0":"patients\\u2019 lives, Pfizer applies a modernized approach to","1":"access, focused on affordability and delivery for patients"},"*":{"0":null,"1":null},"New technologies that reduce barriers to care and digital":{"0":"wallets with the potential to pass rebates directly to","1":"patients at the point of sale"},"Unnamed: 1":{"0":"Product Donation Programs","1":null}}',
 '{"looking at minority versus non-minority pay in the U.S.,":{"0":null,"1":"minorities are at dollar-for-dollar parity (100 percent) with the","2":null,"3":"pay of non-minorities.","4":null,"5":null,"6":"For the second year, Pfizer released median pay gaps for","7":null,"8":"women globally and minorities in the U.S., measuring the","9":"distribution of pay among colleagues without accounting","10":"for any factors. Pfizer\'s pay equity study demonstrated the","11":"median pay for women globally was 101.3 percent of the","12":"median pay of men, and the median pay for minorities in the","13":"U.S. w

In [17]:
from dotenv import load_dotenv
import os
from openai import OpenAI

def ask_llm(esg_category,data,table_data,prompts_func):
    load_dotenv('secret.env')

    API_KEY = os.getenv('API_KEY')

    client = OpenAI(
    base_url="https://openrouter.ai/api/v1",
    api_key=API_KEY,
    )
    jsons = [] #create list to store llm response
    for i in esg_category: #Run LLM 3 Times according to E,S,G to prevent hallucination
        df = data[data['esg_cat'] == i].to_json()
        prompts = prompts_func(df,table_data)
        completion = client.chat.completions.create(
        extra_headers={
            "HTTP-Referer": "<YOUR_SITE_URL>",
            "X-Title": "<YOUR_SITE_NAME>",
        },
        extra_body={},
        model="google/gemini-2.0-pro-exp-02-05:free",
        messages=[
            {
            "role": "user",
            "content": prompts
            },
        ]
        )
        print(completion.choices[0].message.content)
        jsons.append(completion.choices[0].message.content)
    return jsons

In [18]:
output = ask_llm(df2['esg_cat'].unique(),data, tabular, prompts)

```json
[
  {
    "company": "Pfizer",
    "year": 2022,
    "ticker": "PFE",
    "industry": "Pharmaceuticals",
    "esg_category": "Social",
    "esg_subcategory": "Access_And_Affordability",
    "sentence_type": "Qualitative",
    "esg_framework": "GRI, SASB",
    "raw_score": 6,
    "esg_risk_prediction": 0.4,
    "summarized_sentences_data": "Pfizer's commitment to equitable access and pricing is a core ESG priority, extending beyond COVID-19 to encompass a broader range of medicines and vaccines. The company is working with global health leaders to expand access to 1.2 billion people in 45 lower-income countries, addressing barriers such as diagnosis, education, infrastructure, and storage.  This effort underscores that Pfizer vaccines and medicines cannot create positive impacts for patients if those don't have access to them. Pfizer's 'An Accord for a Healthier World' initiative is specifically focused on closing the health equity gap between wealthier nations and lower-income 

In [19]:
import scripts.json_clean

In [ ]:
cleaned_output = [json.loads(i) for i in json_clean(output)]  
cleaned_output #list of list

[[{'company': 'Pfizer',
   'year': 2022,
   'ticker': 'PFE',
   'industry': 'Pharmaceuticals',
   'esg_category': 'Social',
   'esg_subcategory': 'Access_And_Affordability',
   'sentence_type': 'Qualitative',
   'esg_framework': 'GRI, SASB',
   'raw_score': 6,
   'esg_risk_prediction': 0.4,
   'summarized_sentences_data': "Pfizer's commitment to equitable access and pricing is a core ESG priority, extending beyond COVID-19 to encompass a broader range of medicines and vaccines. The company is working with global health leaders to expand access to 1.2 billion people in 45 lower-income countries, addressing barriers such as diagnosis, education, infrastructure, and storage.This effort underscores that Pfizer vaccines and medicines cannot create positive impacts for patients if those don't have access to them. Pfizer's 'An Accord for a Healthier World' initiative is specifically focused on closing the health equity gap between wealthier nations and lower-income countries. In 2022, 1.3 Bil

In [ ]:
company = cleaned_output[0][0][['company']] #take one entry and extract company,year,ticker to combine to esg_bert
company_year = cleaned_output[0][0][['year']]
company_ticker = cleaned_output[0][0][['ticker']]

TypeError: unhashable type: 'list'